In [ ]:
import azureml.core
import string
import os
import shutil

print("SDK version:", azureml.core.VERSION)

# Deployment parameters

In [ ]:
proj_root='pets'
model_version = 1
support_gpu = False

# Environment variables

In [ ]:
subscription_id = os.getenv('SUBSCRIPTION_ID')
resource_group = os.getenv('RESOURCE_GROUP')
workspace_name = os.getenv('WORKSPACE_NAME')
proj_datastore = os.getenv("PROJ_DATASTORE", default = None)
compute_cpu = os.getenv('AML_AKS_CPU', default='akscpu')
compute_gpu = os.getenv('AML_AKS_GPU', default='aksnv6')

# Constants

In [ ]:
DATA_SUBDIR='data'
TFRECORDS_SUBDIR='tfrecords'
MODELS_SUBDIR='models'
PASCAL_LABEL_MAP_FILE = 'pascal_label_map.pbtxt'

# Set up Azure ML environment

In [ ]:
from azureml.core import Workspace, Datastore
from azureml.core.model import Model
from azureml.core.image import ContainerImage, Image
from azureml.core.webservice import Webservice, AksWebservice

ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
if proj_datastore is None:
    ds = ws.get_default_datastore()
else:
    ds = Datastore.get(ws, datastore_name=proj_datastore)
compute_name = compute_gpu if support_gpu else compute_cpu

model_name = proj_root if proj_root.isalnum() else ''.join(ch for ch in proj_root if ch.isalnum())
model = None
models = Model.list(ws, name=model_name)
for m in models:
    if m.version == model_version:
        model = m
        break
if model is None:
    raise ValueError('model {}:{} not found'.format(model_name, model_version))
print("proj_root:{}, model:{}:{}".format(proj_root, model_name, model_version))

pascal_label_map_in_ds = os.path.join(proj_root, TFRECORDS_SUBDIR, PASCAL_LABEL_MAP_FILE)
inference_docker_image = model_name
webservice_name = model_name + 'svc'

# Build the inference image

In [ ]:
# Inference depends on 2 files:
#    1. the model file itself
#    2. pascal_label_map.pbtxt in order to display labels in human-readable format
ds.download(target_path='.',
            prefix=pascal_label_map_in_ds,
            overwrite=True,
            show_progress=True) #this is downloaded to a subfolder
shutil.copy(pascal_label_map_in_ds, '.') #copy to current folder

In [ ]:
# score.py has to load the model, but there's no way to pass in a parameter or set env variable to the image or web service
with open("score.py", "rt") as fin:
    with open("mscore.py", "wt") as fout:
        for line in fin:
            fout.write(line.replace('__REPLACE_MODEL_NAME__', model_name))

In [ ]:
image_config = ContainerImage.image_configuration(
    execution_script = "mscore.py",
    runtime = "python",
    conda_file = "conda_env.yml",
    description = model_name,
    dependencies = ['./', 'utils/'],
    enable_gpu = support_gpu)

image = ContainerImage.create(name = inference_docker_image, 
                              models = [model], 
                              image_config = image_config,
                              workspace = ws
                              )
image.wait_for_creation(show_output=True)
inference_docker_image_version = image.version
print('created image: {}:{}'.format(inference_docker_image, inference_docker_image_version))

# Deploy a new service or updating an existing service

In [ ]:
try: 
    service = Webservice(name = webservice_name, workspace = ws)
except:
    is_new = False
    print('deploy a new service {}'.format(webservice_name))
else:
    is_new = True
    print('update an existing service {}'.format(webservice_name))

# Deploy a new service

In [ ]:
if is_new:
    aks_target = ws.compute_targets[compute_name]
    aks_config = AksWebservice.deploy_configuration(collect_model_data=True, enable_app_insights=True)
    aks_service_name = webservice_name
    service = Webservice.deploy_from_image(workspace = ws, 
                                            name = aks_service_name,
                                            image = image,
                                            deployment_config = aks_config,
                                            deployment_target = aks_target)
    #service.wait_for_deployment(show_output = True)
    print(service.state)

# Or update an existing service

In [ ]:
if not is_new:
    #if we want to use a different image than the one created above
    #image_id = inference_docker_image + ":" + str(inference_docker_image_version)
    #image = Image(workspace = ws, id=image_id)

    service.update(image = image)
    #service.wait_for_deployment(show_output = True)
    print(service.state)

# Test the deployed service

In [ ]:
from azureml.core.webservice import Webservice

# if you want to use another service than the one deployed above
#webservice_name = 'existing_web_service_rather_than_the_one_just_deployed'
#service = Webservice(name = webservice_name, workspace = ws)
# or if you want to use use a different endpoint
#scoring_uri = 'http://localhost:9090/score'
#primary = ''
scoring_uri = service.scoring_uri
primary, secondary = service.get_keys()
enable_auth = True
print(scoring_uri)

In [ ]:
# pick some images for testing
# IMG_1234.jpg
# IMG_4567.jpg
# IMG_7231.JPG

filename = '/mnt/pliu/testimages/IMG_7231.JPG'

In [ ]:
import requests
import base64
import json

with open(filename, "rb") as image_file:
    base64_bytes = base64.b64encode(image_file.read())

base64_string = base64_bytes.decode('utf-8')
data = {"file": base64_string}
input_data = json.dumps(data)
headers = { 'Content-Type':'application/json' }
if enable_auth:
    headers['Authorization']=f'Bearer {primary}'

resp = requests.post(scoring_uri, input_data, headers = headers)
results = resp.text
print(results)

### draw bounding boxes on detections

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from PIL import Image
import numpy as np

In [ ]:
IMAGE_SIZE = (18,12)
plt.figure(figsize=IMAGE_SIZE)

img_np=mpimg.imread(filename)
#plt.imshow(img_np)
img = Image.fromarray(img_np.astype('uint8'),'RGB')
x, y = img.size

In [ ]:
fig,ax = plt.subplots(1)
# Display the image
ax.imshow(img_np)

# draw box and label for each detection 
detections = json.loads(results)
for detect in detections:
    label = detect['label']
    box = detect['bounding_box']
    ymin, xmin, ymax, xmax = box[0], box[1], box[2], box[3]
    topleft_x, topleft_y = x * xmin, y * ymin
    width, height = x * (xmax - xmin), y * (ymax - ymin)
    print('{}: {}, {}, {}, {}'.format(detect['label'], topleft_x, topleft_y, width, height))

    color = 'green' if label.endswith('OK') else 'red'
    rect = patches.Rectangle((topleft_x, topleft_y), width, height, 
                             linewidth=1, edgecolor=color,facecolor='none')

    ax.add_patch(rect)
    plt.text(topleft_x, topleft_y, label, color=color)

plt.show()